Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos um valor práximo de zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random
from itertools import permutations
from funcoes import computaMochila
from funcoes import funçãoObjetivoPopulação_mochila as funçãoObjetivoPopulação
from funcoes import população_cb as criaPopulaçãoInicial
from funcoes import seleçãoRoletaMax as funçãoSeleção
from funcoes import cruzamentoPontoSimples as funçãoCruzamento
from funcoes import mutação_cb as funçãoMutação

## Códigos e discussão



In [2]:
# Constantes:

TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [3]:
# Busca por algoritmo genético

população = criaPopulaçãoInicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_já_visto = float("-inf")
melhor_indivíduo_já_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção:
    fitness = funçãoObjetivoPopulação(população, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES)
    população = funçãoSeleção(população, fitness)

    # Cruzamento:
    pais = população[0::2]
    mães = população[1::2]

    contador = 0

    for pai, mãe in zip(pais, mães):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funçãoCruzamento(pai, mãe)
            população[contador] = filho1
            população[contador + 1] = filho2

        contador = contador + 2

    # Mutação:
    for n in range(len(população)):
        if random.random() <= CHANCE_MUTACAO:
            indivíduo = população[n]
            população[n] = funçãoMutação(indivíduo)

    # Hall da fama:
    fitness = funçãoObjetivoPopulação(população, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES)
    maior_fitness = max(fitness)
    posição = fitness.index(maior_fitness)
    indivíduo = população[posição].copy()
    valor, peso = computaMochila(indivíduo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_já_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_já_visto = maior_fitness
        melhor_indivíduo_já_visto = indivíduo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor indivíduo encontrado:
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_não, item in zip(melhor_indivíduo_já_visto, ORDEM_DOS_NOMES):
    if pega_ou_não == 1:
        print("+", item)
print()
valor_total, peso_total = computaMochila(
    melhor_indivíduo_já_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 15400 | Peso: 12.1
Maior valor: 16900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ energético
+ tablet
+ teclado musical

Com isso, sua mochila terá o valor de 16900 dinheiros e peso de 14.1 unidades de massa.


**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.



## Conclusão

O problema da mochila apresentado e resolvido por algoritmo genético acima, assim como o problema do caixeiro viajante, é um problema de otimização - porém este agora possui uma restrição; o peso da mochila. Ele tem como objetivo encontrar o valor máximo para o conteúdo de uma mochila, que pode ser enchida de diversos objetos. O problema é que o peso total da mochila não pode ultrapassar o limite dado.

Assim como o do caixeiro viajante, este experimento só pode ter seu resultado "oficial" a partir do cálculo de todas as possibilidades possíveis.

Uma das características mais desafiadoras deste problema é a presença da restrição de peso, que exige alguma maneira de penalização para os indivíduos que extrapolarem o limite definido. A estratégia utilizada no código implementado foi a de definir o valor de fitness destes indivíduos como um valor próximo de zero. Outra estratégia que poderia ser usada, apesar de mais custosa e demorada, seria não permitir que um indivíduo ultrapasse esse valor, forçando uma redefinição de todo indivíduo que o ultrapassasse, sorteando novos objetos.

## Playground

